<a href="https://colab.research.google.com/github/afrozmaria07/anomalyDetection-DBSCAN-PID-GAM-PCA/blob/main/Annomaly_Dectection_GAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install interpret
!pip install liac-arff
import os
import pandas as pd
import numpy as np
from scipy.io import loadmat
import arff
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from interpret.glassbox import ExplainableBoostingClassifier  # GA²M
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 45.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 38.7 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
folder_path = '/content/drive/MyDrive/Dataset Anomaly'
dataset_files = ['/content/drive/MyDrive/Dataset Anomaly/annthyroid.mat', '/content/drive/MyDrive/Dataset Anomaly/ecoli.data', '/content/drive/MyDrive/Dataset Anomaly/cardio.mat','/content/drive/MyDrive/Dataset Anomaly/mulcross.arff','/content/drive/MyDrive/Dataset Anomaly/abalone (2).data']

In [ ]:
def load_data(file_path, file_type):
    if file_type == ".data":
        df = pd.read_csv(file_path, header=None, sep=',')  # Adjust separator if necessary
        # Assuming the last column is the label
        X = df.iloc[:, :-1]  # All columns except the last one
        y = df.iloc[:, -1]   # The last column as labels
        return X, y  # Return both features and labels
    elif file_type == ".mat":
        mat_data = loadmat(file_path)
        if 'X' in mat_data and 'y' in mat_data:
            X = pd.DataFrame(mat_data['X'])
            y = pd.Series(mat_data['y'].flatten())  # Convert labels to 1D
            return X, y
        else:
            print(f"Unexpected structure in {file_path}: {mat_data.keys()}")
            return pd.DataFrame(), pd.Series()  # Return empty DataFrame and Series
    elif file_type == ".arff":
        with open(file_path) as f:
            arff_data = arff.load(f)
        data = np.array(arff_data['data'])
        X = pd.DataFrame(data[:, :-1])  # All columns except the last one
        y = pd.Series(data[:, -1])       # The last column as labels
        return X, y  # Return both features and labels
    else:
        print(f"Unsupported file type: {file_type}")
        return pd.DataFrame(), pd.Series()  # Return empty DataFrame and Series

# Preprocess Data
def preprocess_data(X, y=None):
    # Handle non-numeric columns if they exist
    for col in X.columns:
        if X[col].dtype == 'object':  # Check for non-numeric data
            le = LabelEncoder()  # Use label encoding to convert categorical to numeric
            X[col] = le.fit_transform(X[col].astype(str))

    # Handle missing values
    imputer = SimpleImputer(strategy='mean')
    X_imputed = pd.DataFrame(imputer.fit_transform(X))

    # Normalize the data
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed))

    return X_scaled, y

# Load and preprocess all datasets
def load_and_preprocess_all_data(folder_path, dataset_files):
    combined_X = pd.DataFrame()
    combined_y = pd.Series(dtype=np.float64)  # Empty series to store labels

    for file in dataset_files:
        file_path = os.path.join(folder_path, file)
        file_extension = os.path.splitext(file)[-1]

        X, y = load_data(file_path, file_extension)
        if not X.empty and not y.empty:  # Check for non-empty DataFrames/Series
            X_preprocessed, y_preprocessed = preprocess_data(X, y)
            combined_X = pd.concat([combined_X, X_preprocessed], ignore_index=True)
            combined_y = pd.concat([combined_y, y_preprocessed], ignore_index=True)
        else:
            print(f"Skipped file {file} due to empty data.")

    return combined_X, combined_y

# Apply GA²M model (Training Phase)
def train_ga2m(X_train, y_train):
    model = ExplainableBoostingClassifier()  # GA²M model
    model.fit(X_train, y_train)
    return model

# Test the model with PID (Testing Phase)
def pid_test(model, X_test):
    predictions = model.predict(X_test)
    # Assuming anomaly detection is based on the prediction of the model
    return predictions

# Evaluation metrics
def evaluate_model(y_true, y_pred):
    le = LabelEncoder()
    y_true_encoded = le.fit_transform(y_true.astype(str))
    y_pred_encoded = le.transform(y_pred.astype(str))

    accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
    precision = precision_score(y_true_encoded, y_pred_encoded, average='weighted')
    recall = recall_score(y_true_encoded, y_pred_encoded, average='weighted')
    f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

In [ ]:
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")


In [ ]:
#def load_and_preprocess_all_data(folder_path, dataset_files):
 #   combined_data = pd.DataFrame()

  #  for file in dataset_files:
   #     file_path = os.path.join(folder_path, file)
    #    file_extension = os.path.splitext(file)[-1]

     #   df = load_data(file_path, file_extension)
      #  if df is not None:
       #     df = preprocess_data(df)  # Preprocess the data
        #    combined_data = pd.concat([combined_data, df], ignore_index=True)

    #return combined_data

In [ ]:
#def train_ga2m(X_train, y_train):
 #   model = ExplainableBoostingClassifier()  # GA²M model
  #  model.fit(X_train, y_train)
   # return model

In [ ]:
#def pid_test(model, X_test):
 #   predictions = model.predict(X_test)
    # Assuming anomaly detection is based on the prediction of the model
  #  return predictions

In [ ]:
#def evaluate_model(y_true, y_pred):
 #   accuracy = accuracy_score(y_true, y_pred)
  #  precision = precision_score(y_true, y_pred, average='weighted')
   # recall = recall_score(y_true, y_pred, average='weighted')
    #f1 = f1_score(y_true, y_pred, average='weighted')

    #print(f"Accuracy: {accuracy}")
    #print(f"Precision: {precision}")
    #print(f"Recall: {recall}")
    #print(f"F1 Score: {f1}")

In [ ]:
combined_X, combined_y = load_and_preprocess_all_data(folder_path, dataset_files)

# Check the shapes of combined_X and combined_y
print(f"Shape of combined_X: {combined_X.shape}")
print(f"Shape of combined_y: {combined_y.shape}")


<ipython-input-7-358a7931c1b1>:59: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  combined_y = pd.concat([combined_y, y_preprocessed], ignore_index=True)


Skipped file /content/drive/MyDrive/Dataset Anomaly/ecoli.data due to empty data.
Shape of combined_X: (275352, 21)
Shape of combined_y: (275352,)


In [ ]:
if not combined_X.empty and not combined_y.empty:
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(combined_X, combined_y, test_size=0.3, random_state=42)

    # Train the GA²M model
    ga2m_model = train_ga2m(X_train, y_train)

    # Apply PID for testing
    y_pred = pid_test(ga2m_model, X_test)

/usr/local/lib/python3.10/dist-packages/interpret/glassbox/_ebm/_ebm.py:746: UserWarning: Missing values detected. Our visualizations do not currently display missing values. To retain the glassbox nature of the model you need to either set the missing values to an extreme value like -1000 that will be visible on the graphs, or manually examine the missing value score in ebm.term_scores_[term_index][0]
  warn(
/usr/local/lib/python3.10/dist-packages/interpret/glassbox/_ebm/_ebm.py:1014: UserWarning: Detected multiclass problem. Forcing interactions to 0. Multiclass interactions only have local explanations. They are not currently displayed in the global explanation visualizations. Set interactions=0 to disable this warning. If you still want multiclass interactions, this API accepts a list, and the measure_interactions function can be used to detect them.
  warn(


In [ ]:
print(y_test.dtype)
print(y_pred.dtype)
print(y_test.unique())
print(np.unique(y_pred))

object
<U7
['Normal' 9 0.0 'Anomaly' 10 7 15 6 12 11 1.0 8 18 20 13 16 19 21 5 17 14
 4 3 22 26 23]
['0.0' '1.0' '10' '11' '16' '4' '5' '6' '7' '8' '9' 'Anomaly' 'Normal']


In [ ]:
if not combined_X.empty and not combined_y.empty:

    # Evaluate the model
    evaluate_model(y_test, y_pred)
else:
    print("No data loaded. Please check your data files.")

Accuracy: 0.9874585381207176
Precision: 0.9865180538112541
Recall: 0.9874585381207176
F1 Score: 0.9866645272105448


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
